# Modelos de Regresión

### Se presentan una serie de modelos de regresion para predecir el precio de las propiedades.

In [1]:
# librerias
import pandas as pd
import numpy as np
import os
import textwrap
import matplotlib.pyplot as plt
import seaborn as sns
import pickle

In [2]:
#Leer datos
#
data = pd.read_pickle('../data_limpia.pkl')

In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1548 entries, 0 to 1547
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   descripcion              1548 non-null   object 
 1   ubicacion                1548 non-null   object 
 2   banio                    1548 non-null   float64
 3   cantidad_pisos_edificio  1548 non-null   float64
 4   departamentos_por_piso   1548 non-null   float64
 5   piso_departamento        1548 non-null   float64
 6   orientacion              1548 non-null   object 
 7   estacionamiento          1548 non-null   float64
 8   bodega                   1548 non-null   float64
 9   precio_en_pesos          1548 non-null   float64
 10  precio_en_uf             1548 non-null   float64
 11  construido_nuevo         1548 non-null   float64
 12  dormitorio_nuevo         1548 non-null   float64
 13  superficie_total_nuevo   1548 non-null   float64
 14  superficie_util_nuevo   